# Clustering Crypto

In [1]:
# Initial imports
import altair as alt
from vega_datasets import data

import requests
import pandas as pd
#import matplotlib.pyplot as plt
#import hvplot.pandas
#import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import json

### Fetching Cryptocurrency Data

In [13]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
crypto_data=requests.get(url).json()
# crypto_data

In [14]:
crypto_url_df=pd.DataFrame(crypto_data["Data"]).T
crypto_url_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42,0,0,0,blockchain,scrypt,4.54005,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
crypto_url_df.columns

Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'CoinName', 'FullName', 'Description', 'AssetTokenStatus', 'Algorithm',
       'ProofType', 'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
       'IsTrading', 'TotalCoinsMined', 'CirculatingSupply', 'BlockNumber',
       'NetHashesPerSecond', 'BlockReward', 'BlockTime', 'AssetLaunchDate',
       'AssetWhitepaperUrl', 'AssetWebsiteUrl', 'MaxSupply', 'MktCapPenalty',
       'IsUsedInDefi', 'IsUsedInNft', 'PlatformType', 'AlgorithmType',
       'Difficulty', 'BuiltOn', 'SmartContractAddress', 'DecimalPoints'],
      dtype='object')

In [55]:
crypto_df = crypto_url_df[['CoinName', 'Algorithm', 'IsTrading', 'ProofType',
       'TotalCoinsMined', 'MaxSupply']]

In [17]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.



In [8]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

In [9]:
# Create a DataFrame
# crypto_dff=pd.read_csv("crypto_data.csv")
#  crypto_dff.head()

In [13]:
# crypto_dff.columns[1:].values

### Data Preprocessing

In [10]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
# crypto_df=crypto_dff.drop(columns="Unnamed: 0")


In [19]:
#crypto_df=crypto_url_df

In [20]:
#crypto_df = crypto_url_df.copy()

In [56]:
# Keep only cryptocurrencies that are trading
is_Trding = crypto_df['IsTrading']==True


In [57]:
crypto_trading = crypto_df[is_Trding]
crypto_trading.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
MVP,Merculet,N/A,True,N/A,10000000000,-1
ETERNALT,Eternal Token,N/A,True,N/A,NaN,NaN
XET,Xfinite Entertainment Token,N/A,True,N/A,0,4000000000
AXIAL,AXiaL,N/A,True,N/A,NaN,NaN
AXL,AXL INU,N/A,True,N/A,NaN,NaN


In [58]:
[i for i in crypto_df.Algorithm.unique() if i.startswith("N")]

['N/A', 'NIST5', 'NeoScrypt', 'NEP-5', 'NRC20 Token']

In [59]:
# Keep only cryptocurrencies with a working algorithm

crypto_df = crypto_df[crypto_df['Algorithm'] != 'N/A']
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,42
365,365Coin,X11,True,PoW/PoS,0,-1
404,404Coin,Scrypt,True,PoW/PoS,0,-1
611,SixEleven,SHA-256,True,PoW,0,0
808,808,SHA-256,True,PoW/PoS,0,0


In [62]:
# Remove the 'IsTrading' column
crypto_df=crypto_df.drop(columns="IsTrading")
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,42
365,365Coin,X11,PoW/PoS,0,-1
404,404Coin,Scrypt,PoW/PoS,0,-1
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0


In [63]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True,how="any",axis=0)


In [64]:
crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
MaxSupply          0
dtype: int64

In [65]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined']!=0]
crypto_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 343 entries, 42 to KMD
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CoinName         343 non-null    object
 1   Algorithm        343 non-null    object
 2   ProofType        343 non-null    object
 3   TotalCoinsMined  343 non-null    object
 4   MaxSupply        343 non-null    object
dtypes: object(5)
memory usage: 16.1+ KB


In [66]:
# Drop rows where there are 'N/A' text values
# crypto_df[crypto_df.iloc[:] != "N/A"]


In [67]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coin_name=pd.DataFrame(crypto_df["CoinName"], index = crypto_df.index)
coin_name.head()

,CoinName
42,42 Coin
NSR,NuShares
TRI,Triangles Coin
CMTC,CometCoin
CHAT,OpenChat


In [68]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm

crypto_df=crypto_df.drop(columns="CoinName")


In [69]:
crypto_df

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,Scrypt,PoW/PoS,41.999952,42
NSR,PoS,PoS,6173405737.8311,0
TRI,X13,PoW/PoS,191620.842403,0
CMTC,Scrypt,PoW,872830,0
CHAT,Scrypt,PoW/PoS,1000000000,-1
...,...,...,...,...
HPB,DPoS,N/A,101119950,100000000
ZEN,Equihash,PoW,12099612.5,21000000
ICX,Loopchain,PoS,922957379.45422,-1
KCASH,SHA-512,Zero-Knowledge Proof,1000000000,-1


In [70]:
# Create dummy variables for text features
features_df=crypto_df.copy()
X = pd.get_dummies(features_df)
X.head()

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,Algorithm_Autolykos,Algorithm_BEP-2,Algorithm_BEP-2 Token,Algorithm_BEP-20 Token,Algorithm_BEP2 Token,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake2B + SHA3,Algorithm_Blake2b,Algorithm_C31,...,MaxSupply_16555000000.0,MaxSupply_21000000000.0,MaxSupply_40000000000.0,MaxSupply_45000000000.0,MaxSupply_70000000000.0,MaxSupply_86712634466.0,MaxSupply_100000000000.0,MaxSupply_184470000000.0,MaxSupply_1000016730264.435,MaxSupply_21000000000000.0
42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NSR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TRI,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CMTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHAT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
# Standardize data
scaler=StandardScaler().fit(X)


In [72]:
X_scaled=scaler.transform(X)
X_scaled

array([[-0.05407381, -0.09393364, -0.05407381, ..., -0.05407381,
        -0.05407381, -0.05407381],
       [-0.05407381, -0.09393364, -0.05407381, ..., -0.05407381,
        -0.05407381, -0.05407381],
       [-0.05407381, -0.09393364, -0.05407381, ..., -0.05407381,
        -0.05407381, -0.05407381],
       ...,
       [-0.05407381, -0.09393364, -0.05407381, ..., -0.05407381,
        -0.05407381, -0.05407381],
       [-0.05407381, -0.09393364, -0.05407381, ..., -0.05407381,
        -0.05407381, -0.05407381],
       [-0.05407381, -0.09393364, -0.05407381, ..., -0.05407381,
        -0.05407381, -0.05407381]])

### Reducing Dimensions Using PCA

In [73]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
print(X_pca)

[[ 3.08172143 -0.37533282 -0.17918301]
 [ 2.9900899  -0.30181021  0.01241542]
 [ 2.9823289  -0.33598185 -0.1273412 ]
 ...
 [ 0.843971   -0.14791002  0.15205789]
 [ 0.27726939 -0.05763338  0.09720805]
 [ 4.66709819 -0.74377137 -1.0984918 ]]


In [74]:
# Create a DataFrame with the principal components data
pcs_df=pd.DataFrame(data=X_pca,columns=["PC 1", "PC 2","Pc 3"],index=crypto_df.index)
pcs_df.head()

,PC 1,PC 2,Pc 3
42,3.081721,-0.375333,-0.179183
NSR,2.990090,-0.301810,0.012415
TRI,2.982329,-0.335982,-0.127341
CMTC,3.553733,-0.485368,-0.421745
CHAT,0.842223,-0.190679,-0.087000


In [75]:
pca.explained_variance_ratio_

array([0.01158953, 0.00932534, 0.00857964])

### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [39]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [40]:
# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

In [41]:
#df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

In [42]:
import altair as alt
alt.Chart(df_elbow).mark_line().encode(
    x='k',
    y='inertia'
)

alt.Chart(...)

Running K-Means with `k=<4>`

In [43]:
# Initialize the K-Means model
model=KMeans(n_clusters=4,random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions=model.predict(pcs_df)
print(predictions)

[1 1 1 1 1 1 1 1 1 3 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1
 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1
 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 2 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1]


In [44]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["class"]=model.labels_
pcs_df.head()

,PC 1,PC 2,Pc 3,class
42,3.108579,-0.348426,-0.186816,1
NSR,3.059428,-0.282890,0.026326,1
TRI,3.022211,-0.311464,-0.131183,1
CMTC,3.542981,-0.448657,-0.445054,1
CHAT,0.985582,-0.192814,-0.075409,1


In [45]:
clustered_df=pd.concat([crypto_df,coin_name,pcs_df],axis="columns", join="inner")


In [46]:
clustered_df = clustered_df.rename(columns={'MaxSupply': 'TotalCoinSupply'})

In [47]:
clustered_df.tail(20)

,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,PC 1,PC 2,Pc 3,class
NAV,X13,True,PoW/PoS,72883915.858578,-1,NavCoin,1.239020,-0.208676,-0.026842,1
BCN,CryptoNight,True,PoW,184467440737.09552,184470000000,ByteCoin,3.548430,-0.481536,-0.568118,1
CLO,Ethash,True,PoW,3116298217.027898,6500000000,Callisto Network,3.468270,-0.448300,-0.545679,1
ADA,Ouroboros,True,PoS,33041532224.54863,45000000000,Cardano,3.477116,-0.268446,0.437483,1
NXS,SHA3,True,PoW/nPoS,72801490.478337,78000000,Nexus,4.535392,36.801945,-0.347662,2
CLOAK,X13,True,PoW/PoS,5768661.930408,-1,CloakCoin,1.239020,-0.208676,-0.026842,1
DASH,X11,True,PoW/PoSe,10597643.210304,18900000,Dash,3.587057,-0.357058,0.140084,1
DERO,CryptoNight,True,PoW,18400000,-1,Dero,1.220630,-0.279200,-0.317427,1
DGB,Multiple,True,PoW,15120116106.850199,21000000000,DigiByte,3.356695,-0.454456,-0.641920,1
DOGE,Scrypt,True,PoW,133342876383.705231,-1,Dogecoin,1.759791,-0.345869,-0.340714,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [76]:
import altair as alt
from vega_datasets import data

source = clustered_df

alt.Chart(source).mark_circle(size=60).encode(
     x="PC 1",
    y="PC 2",
    color='class',
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)

In [49]:
#!pip install -U altair

#### Table of Tradable Cryptocurrencies

In [50]:
# Table with tradable cryptos


In [51]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [52]:
# Scale data to create the scatter plot



In [53]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
source = clustered_df

alt.Chart(source).mark_circle(size=60).encode(
     x="TotalCoinsMined",
    y="TotalCoinSupply",
   
).interactive()


alt.Chart(...)

In [78]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        clustered_df[
            [
                "CoinName",
                "Algorithm",
                "ProofType",
                "TotalCoinSupply",
                "TotalCoinsMined",
                "class",
            ]
        ]
    )

,CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class
42,42 Coin,Scrypt,PoW/PoS,42,41.999952,1
NSR,NuShares,PoS,PoS,0,6173405737.8311,1
TRI,Triangles Coin,X13,PoW/PoS,0,191620.842403,1
CMTC,CometCoin,Scrypt,PoW,0,872830,1
CHAT,OpenChat,Scrypt,PoW/PoS,-1,1000000000,1
QRL,Quantum Resistant Ledger,RandomX,PoW,105000000,76019638.084719,1
PURA,Pura,X11,PoW,-1,188358976.839698,1
BTCP,Bitcoin Private,Equihash,PoW,22873588,3818878.387802,1
ADK,Aidos Kuneen,IMesh,PoW,0,25000000,1
DAPS,DAPS Coin,Dagger,PoW/PoS/PoA,70000000000,62319462900,3
